# Part 3B Using compression

> Objectives:
> * How to compress chunked datasets
> * Learn how to fine-tune the HDF5 compression pipeline to suit your needs
> * How to use pandas for reading CSV files

In [7]:
import os
import numpy as np
import pandas as pd
import tables

In [8]:
import os
import shutil
data_dir = "compression"
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.mkdir(data_dir)

## Intermezzo: the movielens-1M datasets

Previous work by Greg Reda: http://www.gregreda.com/2013/10/26/using-pandas-on-the-movielens-dataset/

This dataset describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. These files contain 1,000,209 anonymous ratings of approximately 3,900 movies  made by 6,040 MovieLens users who joined MovieLens in 2000.


In [48]:
# Import CSV files via pandas
dset = 'datasets\movielens-1m'
fdata = os.path.join(dset, 'ratings.dat.gz')
fitem = os.path.join(dset, 'movies.dat.gz')

# pass in column names for each CSV
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(fdata, sep=';', names=r_cols)

m_cols = ['movie_id', 'title', 'genres']
movies = pd.read_csv(fitem, sep=';', names=m_cols,
                     dtype={'title': object, 'genres': object})

In [49]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [50]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
movie_id    3883 non-null int64
title       3883 non-null object
genres      3883 non-null object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [46]:
movies.ftypes

movie_id    object:dense
title       object:dense
genres      object:dense
dtype: object

In [47]:
ratings.head()

user_id  movie_id  rating  unix_timestamp
1 NaN 122      NaN       5.0     NaN       838985046
      185      NaN       5.0     NaN       838983525
      231      NaN       5.0     NaN       838983392
      292      NaN       5.0     NaN       838983421
      316      NaN       5.0     NaN       838983392

In [14]:
ratings.ftypes

user_id           int64:dense
movie_id          int64:dense
rating            int64:dense
unix_timestamp    int64:dense
dtype: object

## Storing in HDF5/PyTables in compressed form

In [15]:
def to_hdf5(ratings, movies, filters):
    
    class Ratings(tables.IsDescription):
        user_id = tables.Int32Col(pos=0)
        movie_id = tables.Int32Col(pos=1)
        rating = tables.Int8Col(pos=2)
        unix_timestamp = tables.Int64Col(pos=3)
    
    class Movies(tables.IsDescription):
        movie_id = tables.Int32Col(pos=0)
        title = tables.StringCol(100, pos=1)
        genres = tables.StringCol(50, pos=2)
    
    def get_filename(filters):
        if filters.complevel != 0:
            complib = filters.complib if ":" not in filters.complib else filters.complib.replace(":", "-")
            shuffle = "shuffle" if filters.shuffle else "noshuffle"
            filename = "%s/%s-%d-%s.h5" % (data_dir, complib, filters.complevel, shuffle)
        else:
            filename = "%s/no-compression.h5" % (data_dir,)
        return filename

    filename = get_filename(filters)
    print("Creating file:", filename)
    with tables.open_file(filename, "w") as f:
        table_ratings = f.create_table(f.root, "ratings", Ratings, filters=filters, expectedrows=len(ratings))
        table_ratings.append([ratings[col].values for col in ratings.ftypes.keys()])
        table_movies = f.create_table(f.root, "movies", Movies, filters=filters, expectedrows=len(movies))
        table_movies.append([movies[col].values for col in movies.ftypes.keys()])
    return filename

In [16]:
%%time
filters = tables.Filters(complevel=5)
h5file = to_hdf5(ratings, movies, filters)

Creating file: compression/zlib-5-shuffle.h5
Wall time: 639 ms


In [17]:
!ptdump -v {h5file}

/ (RootGroup) ''
/movies (Table(3883,), shuffle, zlib(5)) ''
  description := {
  "movie_id": Int32Col(shape=(), dflt=0, pos=0),
  "title": StringCol(itemsize=100, shape=(), dflt=b'', pos=1),
  "genres": StringCol(itemsize=50, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (425,)
/ratings (Table(1000209,), shuffle, zlib(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "movie_id": Int32Col(shape=(), dflt=0, pos=1),
  "rating": Int8Col(shape=(), dflt=0, pos=2),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=3)}
  byteorder := 'little'
  chunkshape := (7710,)


### Exercise 1

PyTables comes with out-of-box support for a series of codecs.  Do a quick comparison between "zlib", "bzip2", and "blosc" for compression levels of 1 (fastest), 5 and 9 (slowest).  Which one compresses best?  Which one compresses faster?

Also, Blosc being a meta-compressor, it has support for different codecs internally that can be selected from PyTables in the "blosc:`codec`" form.  Do another comparison between internal Blosc codecs, namely, "blosc:blosclz" (the default), "blosc:lz4", "blosc:lz4hc", "blosc:snappy", "blosc:zlib" and "blosc:zstd".

Finally, avoid any compression totally (`complevel=0`).  How fast it is compared with existing codecs?

In [18]:
for complib in ("zlib", "bzip2", "blosc"):
    filters = tables.Filters(complevel=5, complib=complib)
    %time to_hdf5(ratings, movies, filters)

Creating file: compression/zlib-5-shuffle.h5
Wall time: 640 ms
Creating file: compression/zlib-5-shuffle.h5


C:\Miniconda3\envs\py36\lib\site-packages\tables\filters.py:331: FiltersWarning: compression library ``bzip2`` is not available; using ``zlib`` instead
  % (complib, default_complib), FiltersWarning)


Wall time: 609 ms
Creating file: compression/blosc-5-shuffle.h5
Wall time: 131 ms


In [19]:
!ls -lh {data_dir}

total 9.3M
-rw-r--r-- 1 tomkooij 197613 5.0M Jun 19 10:58 blosc-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 4.3M Jun 19 10:58 zlib-5-shuffle.h5


In [20]:
for complib in ("blosc:blosclz", "blosc:lz4", "blosc:lz4hc", "blosc:snappy", "blosc:zlib", "blosc:zstd"):
    filters = tables.Filters(complevel=5, complib=complib)
    %time to_hdf5(ratings, movies, filters)

Creating file: compression/blosc-blosclz-5-shuffle.h5
Wall time: 148 ms
Creating file: compression/blosc-lz4-5-shuffle.h5
Wall time: 94.1 ms
Creating file: compression/blosc-lz4hc-5-shuffle.h5
Wall time: 331 ms
Creating file: compression/blosc-snappy-5-shuffle.h5
Wall time: 115 ms
Creating file: compression/blosc-zlib-5-shuffle.h5
Wall time: 664 ms
Creating file: compression/blosc-zstd-5-shuffle.h5
Wall time: 559 ms


In [21]:
# Finally, the uncompressed case
filters = tables.Filters(complevel=0)
%time to_hdf5(ratings, movies, filters)

Creating file: compression/no-compression.h5
Wall time: 86 ms


'compression/no-compression.h5'

In [22]:
!ls -lh {data_dir}

total 56M
-rw-r--r-- 1 tomkooij 197613 5.0M Jun 19 10:58 blosc-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 5.0M Jun 19 10:58 blosc-blosclz-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 5.4M Jun 19 10:58 blosc-lz4-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 4.8M Jun 19 10:58 blosc-lz4hc-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 5.5M Jun 19 10:58 blosc-snappy-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 4.4M Jun 19 10:58 blosc-zlib-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 4.3M Jun 19 10:58 blosc-zstd-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613  17M Jun 19 10:58 no-compression.h5
-rw-r--r-- 1 tomkooij 197613 4.3M Jun 19 10:58 zlib-5-shuffle.h5


## Reading compressed datasets

In [23]:
files = list(os.walk(data_dir))[0][2]

In [24]:
files

['blosc-5-shuffle.h5',
 'blosc-blosclz-5-shuffle.h5',
 'blosc-lz4-5-shuffle.h5',
 'blosc-lz4hc-5-shuffle.h5',
 'blosc-snappy-5-shuffle.h5',
 'blosc-zlib-5-shuffle.h5',
 'blosc-zstd-5-shuffle.h5',
 'no-compression.h5',
 'zlib-5-shuffle.h5']

In [25]:
for f in files:
    print("Reading file:", f)
    with tables.open_file(os.path.join(data_dir, f)) as h5f:
        %time h5f.root.ratings[:]

Reading file: blosc-5-shuffle.h5
Wall time: 56 ms
Reading file: blosc-blosclz-5-shuffle.h5
Wall time: 41 ms
Reading file: blosc-lz4-5-shuffle.h5
Wall time: 32 ms
Reading file: blosc-lz4hc-5-shuffle.h5
Wall time: 32 ms
Reading file: blosc-snappy-5-shuffle.h5
Wall time: 52.2 ms
Reading file: blosc-zlib-5-shuffle.h5
Wall time: 107 ms
Reading file: blosc-zstd-5-shuffle.h5
Wall time: 58 ms
Reading file: no-compression.h5
Wall time: 19 ms
Reading file: zlib-5-shuffle.h5
Wall time: 125 ms


### Exercise 2

Which codec and compression level can read the fastest?  How it does compare with reading an uncompressed dataset?

### Exercise 3

Blosc can use multithreading for compressing/decompressing, although it is disabled by default.  You can enable a multithreaded Blosc in a series of ways, but perhaps the easiest is to set the "BLOSC_NTHREADS" environment variable to the desired number of threads (typically the available number of cores in your computer).

Execute the cell below and re-do the reading benchmarks and look at how the reading speed vary.  Pay special attention to the difference between the CPU times and wall times.

In [26]:
os.environ["BLOSC_NTHREADS"] = "4"  # set to any other number you prefer

In [40]:
for f in files:
    for nthreads in [1, 2, 4]:
        os.environ["BLOSC_NTHREADS"] = "%s" % nthreads
        print("Reading file:", f, 'nthreads=', os.environ["BLOSC_NTHREADS"])
        with tables.open_file(os.path.join(data_dir, f)) as h5f:
            %time h5f.root.ratings[:]

Reading file: blosc-5-shuffle.h5 nthreads= 1
Wall time: 52 ms
Reading file: blosc-5-shuffle.h5 nthreads= 2
Wall time: 48.1 ms
Reading file: blosc-5-shuffle.h5 nthreads= 4
Wall time: 41.6 ms
Reading file: blosc-blosclz-5-shuffle.h5 nthreads= 1
Wall time: 41 ms
Reading file: blosc-blosclz-5-shuffle.h5 nthreads= 2
Wall time: 53 ms
Reading file: blosc-blosclz-5-shuffle.h5 nthreads= 4
Wall time: 43.2 ms
Reading file: blosc-lz4-5-shuffle.h5 nthreads= 1
Wall time: 31 ms
Reading file: blosc-lz4-5-shuffle.h5 nthreads= 2
Wall time: 33.7 ms
Reading file: blosc-lz4-5-shuffle.h5 nthreads= 4
Wall time: 38.8 ms
Reading file: blosc-lz4hc-5-shuffle.h5 nthreads= 1
Wall time: 39 ms
Reading file: blosc-lz4hc-5-shuffle.h5 nthreads= 2
Wall time: 30 ms
Reading file: blosc-lz4hc-5-shuffle.h5 nthreads= 4
Wall time: 36.2 ms
Reading file: blosc-snappy-5-shuffle.h5 nthreads= 1
Wall time: 40 ms
Reading file: blosc-snappy-5-shuffle.h5 nthreads= 2
Wall time: 46.4 ms
Reading file: blosc-snappy-5-shuffle.h5 nthreads= 